# Linear Regression

The following code uses Tensowflow's Estimator API to train a linear regression model. Code adapted from Tensorflow's linear regression tutorial: https://github.com/tensorflow/docs/blob/master/site/en/tutorials/estimator/linear.ipynb.

In [1]:
from itertools import combinations
import numpy as np
import os, sys
import pandas as pd
from pathlib import Path
from pi_pact_sort import categorize, bin_categorize
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc

DROP_COLUMNS = ['ADDRESS', 'TIMESTAMP', 'UUID', 'MAJOR', 'MINOR', 'TX POWER', 'TEMPERATURE',
                'PITCH', 'ROLL', 'YAW', 'SCAN']
FEATURE_COLUMNS = ['RSSI', 'HUMIDITY', 'PRESSURE']
SAMPLE_SIZE = 30000
np.random.seed(0)
tf.random.set_seed(1)

# Initialize DataFrame
data: pd.DataFrame = pd.DataFrame(columns=['RSSI', 'DISTANCE', 'HUMIDITY', 'PRESSURE'])
data_copy: pd.DataFrame = data.copy()
csv_file: Path
for csv_file in Path('.').glob('indoor-noObstruct-SenseHat*/*.csv'):
    datapart: pd.DataFrame = pd.read_csv(csv_file)
    for column in DROP_COLUMNS:
        if column in datapart.columns:
            datapart = datapart.drop([column], 1)
    data_copy = data_copy.append(datapart)

# Categorize distance
data_copy['DISTANCE'] = data_copy['DISTANCE'].map(categorize)

# Sample data from each distance category
for value in data_copy['DISTANCE'].unique():
    datapart = data_copy[data_copy.DISTANCE == value]
    datapart = datapart.sample(SAMPLE_SIZE, random_state=1)
    data = data.append(datapart)

# Assign features and labels
X: pd.DataFrame = data.astype({'RSSI': 'float32', 'DISTANCE': 'int32',
                               'HUMIDITY': 'float32', 'PRESSURE': 'float32'})
y: pd.Series = X.pop('DISTANCE')
    

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Code below adapted from Tensorflow's "Build a linear model with Estimators" tutorial:
# https://github.com/tensorflow/docs/blob/master/site/en/tutorials/estimator/linear.ipynb
feature_columns = []
derived_columns = []
for feature_name in FEATURE_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name))

for feature1_name, feature2_name in combinations(FEATURE_COLUMNS, 2):
    derived_columns.append(tf.feature_column.crossed_column([feature1_name, feature2_name], hash_bucket_size=1000))

def make_input_fn(data_df, label_df, num_epochs=100, shuffle=True, batch_size=64):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000, seed=1)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

train_input_fn = make_input_fn(X_train, y_train)
eval_input_fn = make_input_fn(X_test, y_test, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns+derived_columns,
                                           n_classes=3)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Winst\\AppData\\Local\\Temp\\tmptffqeszu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initializ

INFO:tensorflow:global_step/sec: 1044.45
INFO:tensorflow:loss = 0.27125037, step = 5700 (0.095 sec)
INFO:tensorflow:global_step/sec: 1023.13
INFO:tensorflow:loss = 0.42228556, step = 5800 (0.098 sec)
INFO:tensorflow:global_step/sec: 1165.9
INFO:tensorflow:loss = 0.28260103, step = 5900 (0.087 sec)
INFO:tensorflow:global_step/sec: 1179.62
INFO:tensorflow:loss = 0.25839588, step = 6000 (0.085 sec)
INFO:tensorflow:global_step/sec: 1165.89
INFO:tensorflow:loss = 0.3101523, step = 6100 (0.085 sec)
INFO:tensorflow:global_step/sec: 1078.15
INFO:tensorflow:loss = 0.1589253, step = 6200 (0.093 sec)
INFO:tensorflow:global_step/sec: 1044.44
INFO:tensorflow:loss = 0.34128916, step = 6300 (0.096 sec)
INFO:tensorflow:global_step/sec: 1089.87
INFO:tensorflow:loss = 0.19374937, step = 6400 (0.092 sec)
INFO:tensorflow:global_step/sec: 1066.67
INFO:tensorflow:loss = 0.18051398, step = 6500 (0.094 sec)
INFO:tensorflow:global_step/sec: 1044.44
INFO:tensorflow:loss = 0.20681557, step = 6600 (0.096 sec)
INF

INFO:tensorflow:global_step/sec: 1089.88
INFO:tensorflow:loss = 0.17713724, step = 13900 (0.092 sec)
INFO:tensorflow:global_step/sec: 1114.08
INFO:tensorflow:loss = 0.1295174, step = 14000 (0.089 sec)
INFO:tensorflow:global_step/sec: 1139.39
INFO:tensorflow:loss = 0.2761558, step = 14100 (0.089 sec)
INFO:tensorflow:global_step/sec: 983.018
INFO:tensorflow:loss = 0.40986386, step = 14200 (0.101 sec)
INFO:tensorflow:global_step/sec: 1114.08
INFO:tensorflow:loss = 0.11444037, step = 14300 (0.090 sec)
INFO:tensorflow:global_step/sec: 1179.61
INFO:tensorflow:loss = 0.25499848, step = 14400 (0.086 sec)
INFO:tensorflow:global_step/sec: 1165.91
INFO:tensorflow:loss = 0.20169844, step = 14500 (0.085 sec)
INFO:tensorflow:global_step/sec: 1179.62
INFO:tensorflow:loss = 0.30941418, step = 14600 (0.085 sec)
INFO:tensorflow:global_step/sec: 1139.39
INFO:tensorflow:loss = 0.23074102, step = 14700 (0.088 sec)
INFO:tensorflow:global_step/sec: 1193.66
INFO:tensorflow:loss = 0.2760678, step = 14800 (0.08

INFO:tensorflow:loss = 0.30370983, step = 22000 (0.085 sec)
INFO:tensorflow:global_step/sec: 1044.45
INFO:tensorflow:loss = 0.2005559, step = 22100 (0.096 sec)
INFO:tensorflow:global_step/sec: 1044.44
INFO:tensorflow:loss = 0.110761695, step = 22200 (0.097 sec)
INFO:tensorflow:global_step/sec: 945.92
INFO:tensorflow:loss = 0.17543104, step = 22300 (0.106 sec)
INFO:tensorflow:global_step/sec: 945.922
INFO:tensorflow:loss = 0.3739313, step = 22400 (0.106 sec)
INFO:tensorflow:global_step/sec: 1126.6
INFO:tensorflow:loss = 0.26582545, step = 22500 (0.088 sec)
INFO:tensorflow:global_step/sec: 1101.84
INFO:tensorflow:loss = 0.21380326, step = 22600 (0.091 sec)
INFO:tensorflow:global_step/sec: 1165.89
INFO:tensorflow:loss = 0.19270653, step = 22700 (0.087 sec)
INFO:tensorflow:global_step/sec: 1152.5
INFO:tensorflow:loss = 0.18841454, step = 22800 (0.086 sec)
INFO:tensorflow:global_step/sec: 1152.5
INFO:tensorflow:loss = 0.14619766, step = 22900 (0.088 sec)
INFO:tensorflow:global_step/sec: 117

INFO:tensorflow:global_step/sec: 1179.63
INFO:tensorflow:loss = 0.21203953, step = 30200 (0.085 sec)
INFO:tensorflow:global_step/sec: 1101.84
INFO:tensorflow:loss = 0.2745812, step = 30300 (0.092 sec)
INFO:tensorflow:global_step/sec: 1055.44
INFO:tensorflow:loss = 0.44092035, step = 30400 (0.095 sec)
INFO:tensorflow:global_step/sec: 1023.13
INFO:tensorflow:loss = 0.24078187, step = 30500 (0.097 sec)
INFO:tensorflow:global_step/sec: 1044.46
INFO:tensorflow:loss = 0.32115567, step = 30600 (0.096 sec)
INFO:tensorflow:global_step/sec: 1066.66
INFO:tensorflow:loss = 0.1469765, step = 30700 (0.095 sec)
INFO:tensorflow:global_step/sec: 1023.15
INFO:tensorflow:loss = 0.23357949, step = 30800 (0.098 sec)
INFO:tensorflow:global_step/sec: 1033.68
INFO:tensorflow:loss = 0.3565566, step = 30900 (0.097 sec)
INFO:tensorflow:global_step/sec: 1044.45
INFO:tensorflow:loss = 0.23259556, step = 31000 (0.096 sec)
INFO:tensorflow:global_step/sec: 1139.4
INFO:tensorflow:loss = 0.2107967, step = 31100 (0.087 

...

INFO:tensorflow:global_step/sec: 1179.61
INFO:tensorflow:loss = 0.1705665, step = 87300 (0.085 sec)
INFO:tensorflow:global_step/sec: 1222.77
INFO:tensorflow:loss = 0.21600825, step = 87400 (0.082 sec)
INFO:tensorflow:global_step/sec: 1179.62
INFO:tensorflow:loss = 0.25537816, step = 87500 (0.084 sec)
INFO:tensorflow:global_step/sec: 1179.61
INFO:tensorflow:loss = 0.32579818, step = 87600 (0.085 sec)
INFO:tensorflow:global_step/sec: 1193.67
INFO:tensorflow:loss = 0.32628715, step = 87700 (0.084 sec)
INFO:tensorflow:global_step/sec: 1193.66
INFO:tensorflow:loss = 0.27005887, step = 87800 (0.085 sec)
INFO:tensorflow:global_step/sec: 1237.86
INFO:tensorflow:loss = 0.22616483, step = 87900 (0.080 sec)
INFO:tensorflow:global_step/sec: 1222.76
INFO:tensorflow:loss = 0.19577144, step = 88000 (0.083 sec)
INFO:tensorflow:global_step/sec: 1237.88
INFO:tensorflow:loss = 0.2607981, step = 88100 (0.080 sec)
INFO:tensorflow:global_step/sec: 1237.86
INFO:tensorflow:loss = 0.27889603, step = 88200 (0.0

INFO:tensorflow:loss = 0.19889344, step = 95400 (0.084 sec)
INFO:tensorflow:global_step/sec: 1253.37
INFO:tensorflow:loss = 0.15374836, step = 95500 (0.081 sec)
INFO:tensorflow:global_step/sec: 1253.34
INFO:tensorflow:loss = 0.28173763, step = 95600 (0.080 sec)
INFO:tensorflow:global_step/sec: 1253.33
INFO:tensorflow:loss = 0.20802338, step = 95700 (0.080 sec)
INFO:tensorflow:global_step/sec: 1193.67
INFO:tensorflow:loss = 0.23370901, step = 95800 (0.083 sec)
INFO:tensorflow:global_step/sec: 1253.34
INFO:tensorflow:loss = 0.20215386, step = 95900 (0.081 sec)
INFO:tensorflow:global_step/sec: 1269.21
INFO:tensorflow:loss = 0.17988771, step = 96000 (0.078 sec)
INFO:tensorflow:global_step/sec: 1285.5
INFO:tensorflow:loss = 0.24211363, step = 96100 (0.078 sec)
INFO:tensorflow:global_step/sec: 1208.03
INFO:tensorflow:loss = 0.2670067, step = 96200 (0.084 sec)
INFO:tensorflow:global_step/sec: 1253.32
INFO:tensorflow:loss = 0.23150924, step = 96300 (0.080 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:loss = 0.11210685, step = 103500 (0.087 sec)
INFO:tensorflow:global_step/sec: 1114.08
INFO:tensorflow:loss = 0.19757268, step = 103600 (0.090 sec)
INFO:tensorflow:global_step/sec: 1139.39
INFO:tensorflow:loss = 0.30901635, step = 103700 (0.088 sec)
INFO:tensorflow:global_step/sec: 1044.46
INFO:tensorflow:loss = 0.2384259, step = 103800 (0.096 sec)
INFO:tensorflow:global_step/sec: 1066.67
INFO:tensorflow:loss = 0.19981879, step = 103900 (0.095 sec)
INFO:tensorflow:global_step/sec: 1222.76
INFO:tensorflow:loss = 0.1896213, step = 104000 (0.082 sec)
INFO:tensorflow:global_step/sec: 1237.87
INFO:tensorflow:loss = 0.21001193, step = 104100 (0.081 sec)
INFO:tensorflow:global_step/sec: 1222.78
INFO:tensorflow:loss = 0.22511071, step = 104200 (0.081 sec)
INFO:tensorflow:global_step/sec: 1193.65
INFO:tensorflow:loss = 0.28542593, step = 104300 (0.085 sec)
INFO:tensorflow:global_step/sec: 1222.77
INFO:tensorflow:loss = 0.26245156, step = 104400 (0.081 sec)
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 1269.21
INFO:tensorflow:loss = 0.20581, step = 111600 (0.079 sec)
INFO:tensorflow:global_step/sec: 1126.58
INFO:tensorflow:loss = 0.24349615, step = 111700 (0.089 sec)
INFO:tensorflow:global_step/sec: 1302.2
INFO:tensorflow:loss = 0.13852203, step = 111800 (0.078 sec)
INFO:tensorflow:global_step/sec: 1285.48
INFO:tensorflow:loss = 0.24098313, step = 111900 (0.078 sec)
INFO:tensorflow:global_step/sec: 1269.21
INFO:tensorflow:loss = 0.15385902, step = 112000 (0.079 sec)
INFO:tensorflow:global_step/sec: 1302.16
INFO:tensorflow:loss = 0.15151133, step = 112100 (0.076 sec)
INFO:tensorflow:global_step/sec: 1126.61
INFO:tensorflow:loss = 0.24188116, step = 112200 (0.089 sec)
INFO:tensorflow:global_step/sec: 1269.21
INFO:tensorflow:loss = 0.22782898, step = 112300 (0.079 sec)
INFO:tensorflow:global_step/sec: 1066.67
INFO:tensorflow:loss = 0.14457582, step = 112400 (0.094 sec)
INFO:tensorflow:global_step/sec: 1055.45
INFO:tensorflow:loss = 0.2335794, step = 1125

INFO:tensorflow:global_step/sec: 1023.14
INFO:tensorflow:loss = 0.21629448, step = 119700 (0.097 sec)
INFO:tensorflow:global_step/sec: 1078.14
INFO:tensorflow:loss = 0.24151896, step = 119800 (0.094 sec)
INFO:tensorflow:global_step/sec: 1285.48
INFO:tensorflow:loss = 0.25460202, step = 119900 (0.077 sec)
INFO:tensorflow:global_step/sec: 1179.61
INFO:tensorflow:loss = 0.2086423, step = 120000 (0.085 sec)
INFO:tensorflow:global_step/sec: 1319.31
INFO:tensorflow:loss = 0.21162972, step = 120100 (0.076 sec)
INFO:tensorflow:global_step/sec: 1319.31
INFO:tensorflow:loss = 0.106207296, step = 120200 (0.076 sec)
INFO:tensorflow:global_step/sec: 1179.58
INFO:tensorflow:loss = 0.3083598, step = 120300 (0.086 sec)
INFO:tensorflow:global_step/sec: 726.585
INFO:tensorflow:loss = 0.19344464, step = 120400 (0.139 sec)
INFO:tensorflow:global_step/sec: 696.295
INFO:tensorflow:loss = 0.1435276, step = 120500 (0.142 sec)
INFO:tensorflow:global_step/sec: 783.345
INFO:tensorflow:loss = 0.3282521, step = 12

In [2]:
print(result)

{'accuracy': 0.8987778, 'average_loss': 0.22577944, 'loss': 0.22599071, 'global_step': 126600}


In [3]:
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
    tf.feature_column.make_parse_example_spec(set(feature_columns)))
linear_est.export_saved_model(str(Path('./linear-models/4var-3b-linear-model')), serving_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from C:\Users\Winst\AppData\Local\Temp\tmptffqeszu\model.ckpt-126600
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: linear-models\4var-3b-linear-model\temp-1595052766\saved_model.pb


b'linear-models\\4var-3b-linear-model\\1595052766'

```
import matplotlib.pyplot as plt
pred_dicts = list(linear_est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')
```

```
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)
plt.savefig(str(Path('./linear-models/2var-binary-linear-model/roc_curve.png')))
```